In [1]:
import pandas as pd

# Import dataset
services = pd.read_csv('Aurora_Office_Fase_2_-_all_versions_-_False_-_2023-09-20-21-01-20.csv', sep=';', index_col = False)
services.head()


C:\Users\maola\AppData\Local\Temp\ipykernel_11836\3977090540.py:4: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  services = pd.read_csv('Aurora_Office_Fase_2_-_all_versions_-_False_-_2023-09-20-21-01-20.csv', sep=';', index_col = False)


,logo,Objetivo,Esta_tarea_le_pedir_er_a_estas_preguntas,encuestador,nota2,fecha,coordenadas,_coordenadas_latitude,_coordenadas_longitude,_coordenadas_altitude,...,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,__version__,_tags,_index
0,NaN,NaN,NaN,Ricardo Soler,NaN,2023-09-15,4.724925 -73.96454 2680.0198380947113 6.952901...,4.724925,-73.964540,2680.019838,...,479943570,92f8ae42-e01b-4c5f-880a-9e3f8d910c96,2023-09-16 00:28:06,NaN,NaN,submitted_via_web,NaN,vK47oDotYwKszvMfjHLoFb,NaN,1
1,NaN,NaN,NaN,Ricardo Soler,NaN,2023-09-16,4.724925 -73.96454 2680.022540807724 6.9529018...,4.724925,-73.964540,2680.022541,...,480149964,084c507d-c8be-4a2d-b20e-3ac0bf1b3639,2023-09-16 14:29:38,NaN,NaN,submitted_via_web,NaN,vHWtoEvjjXPrdWm7o4icWV,NaN,2
2,NaN,NaN,NaN,Daniela Carrasco,NaN,2023-09-18,4.64715 -74.08494 2579.5 31.34600067138672,4.647150,-74.084940,2579.500000,...,480890552,02d58eb0-4ce7-4a5b-be4b-05d2e6d40cc0,2023-09-18 15:02:13,NaN,NaN,submitted_via_web,NaN,vHRFqgUkQ6N6KvmLtrtJAi,NaN,3
3,NaN,NaN,NaN,Maria Olarte,NaN,2023-09-18,7.900174 -72.507584 0 0,7.900174,-72.507584,0.000000,...,480891189,5bfe7631-ce12-4b3a-9194-34dfbe6f9db4,2023-09-18 15:03:43,NaN,NaN,submitted_via_web,NaN,vHRFqgUkQ6N6KvmLtrtJAi,NaN,4


In [3]:
# rename variables
# Are still missing the ones related to schedule
services_carto = services.rename(columns={'_coordenadas_latitude': 'lat', '_coordenadas_longitude' : 'long',  'pais' : 'ubicacion' , 'punto_reporte' : 'lugar_enc'
                                          , 'ubicacion' : 'punto_ayud' , 'cualotropunto' : 'punto_ayud_otro' , 'infraestructura' : 'cuenta_con' , 'sexo' : 'identifica'
                                          , 'organizacionpertence' : 'org_pert', 'infraestructuradiscapacidad' : 'serv_disca' , 'servicios' : 'serv_tipo'  })
services_carto.head()

,logo,Objetivo,Esta_tarea_le_pedir_er_a_estas_preguntas,encuestador,nota2,fecha,coordenadas,lat,long,_coordenadas_altitude,...,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,__version__,_tags,_index
0,NaN,NaN,NaN,Ricardo Soler,NaN,2023-09-15,4.724925 -73.96454 2680.0198380947113 6.952901...,4.724925,-73.964540,2680.019838,...,479943570,92f8ae42-e01b-4c5f-880a-9e3f8d910c96,2023-09-16 00:28:06,NaN,NaN,submitted_via_web,NaN,vK47oDotYwKszvMfjHLoFb,NaN,1
1,NaN,NaN,NaN,Ricardo Soler,NaN,2023-09-16,4.724925 -73.96454 2680.022540807724 6.9529018...,4.724925,-73.964540,2680.022541,...,480149964,084c507d-c8be-4a2d-b20e-3ac0bf1b3639,2023-09-16 14:29:38,NaN,NaN,submitted_via_web,NaN,vHWtoEvjjXPrdWm7o4icWV,NaN,2
2,NaN,NaN,NaN,Daniela Carrasco,NaN,2023-09-18,4.64715 -74.08494 2579.5 31.34600067138672,4.647150,-74.084940,2579.500000,...,480890552,02d58eb0-4ce7-4a5b-be4b-05d2e6d40cc0,2023-09-18 15:02:13,NaN,NaN,submitted_via_web,NaN,vHRFqgUkQ6N6KvmLtrtJAi,NaN,3
3,NaN,NaN,NaN,Maria Olarte,NaN,2023-09-18,7.900174 -72.507584 0 0,7.900174,-72.507584,0.000000,...,480891189,5bfe7631-ce12-4b3a-9194-34dfbe6f9db4,2023-09-18 15:03:43,NaN,NaN,submitted_via_web,NaN,vHRFqgUkQ6N6KvmLtrtJAi,NaN,4


In [ ]:
# serv_tipo id the variable to test (separating by pipe symbol and categorized with number)
# Asesoría legal : 1
# Alimentación y nutrición : 2
# Salud, primeros auxilios o atención médica: 3
# Agua potable: 4
# Saneamiento: 5
# Higiene: 6
# Educación: 7
# Alojamiento temporal: 8
# Servicios de protección : 9
# Acompañamiento o apoyo psicosocial: 10
# Transporte humanitario: 11
# Restablecimiento de contacto familiar : 12
# Transferencias monetarias: 13
# Otro: 14
#
